<a href="https://colab.research.google.com/github/tdopsmang/Colab-projects/blob/main/Servicing_km_V1_0_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Did not work on RR path, rather added bus details upto DP, in consolidated report, also batch processing done in few cases

In [16]:
!pip install --upgrade gspread
!pip install gspread google-auth-oauthlib google-auth-httplib2
!pip install gspread google-auth oauth2client
!pip install gspread oauth2client pandas matplotlib seaborn

In [17]:
from google.colab import auth
auth.authenticate_user()
import pandas as pd
import gspread
from google.auth import default
creds, _ = default()
from google.oauth2.service_account import Credentials
gc = gspread.authorize(creds)
from oauth2client.client import GoogleCredentials
from oauth2client.service_account import ServiceAccountCredentials
from datetime import datetime
from collections import defaultdict

In [18]:
# Details our Database spreadsheet which has details of all files, ID in Transport Department

Database_File_spreadsheet_ID = gc.open_by_key('1nJKyvV1WQmZzvbjOP7Hsp1bQJmiNsYoJOH_jIL7H9PI') #ID of Database Spreadsheet
MASsheetID = Database_File_spreadsheet_ID.worksheet('MAS')                                    #MAS Worksheet
OdometersheetID = Database_File_spreadsheet_ID.worksheet('Odometer')                          #Odometer Worksheet

#STS RR SVP
RR_Oil_Lub_Coolant_DEF_spreadsheet_id = MASsheetID.acell('C2').value                  # Get the spreadsheet ID from cell C2
RR_Oil_Lub = gc.open_by_key(RR_Oil_Lub_Coolant_DEF_spreadsheet_id)                    # Open RR_Oil_Lub spreadsheet

RR_Engine_oil_CH_BSIV_worksheet_name = MASsheetID.acell('B2').value                        # Get the worksheet name from cell B2
engine_oil_CH_BSIV_CH_MAS_rr = RR_Oil_Lub.worksheet(RR_Engine_oil_CH_BSIV_worksheet_name)  # Open RR CH engineoil worksheet

RR_Engine_oil_CK_BSVI_worksheet_name = MASsheetID.acell('B3').value                        # Get the worksheet name from cell B3
engine_oil_CK_BSVI_CK_MAS_rr = RR_Oil_Lub.worksheet(RR_Engine_oil_CK_BSVI_worksheet_name)  # Open RR CK engineoil worksheet

RR_Report1_worksheet_name = MASsheetID.acell('B8').value                                # Get the worksheet name from report1, RR Km between and after last servicing
RR_Report1_Worksheet = RR_Oil_Lub.worksheet(RR_Report1_worksheet_name)                  # Open RR Km between and after last servicing worksheet

#STS ATR SVP
ATR_Oil_Lub_Coolant_DEF_spreadsheet_id = MASsheetID.acell('C52').value                  # Get the spreadsheet ID from cell C51 ATR
ATR_Oil_Lub = gc.open_by_key(ATR_Oil_Lub_Coolant_DEF_spreadsheet_id)                    # Open ATR_Oil_Lub spreadsheet

ATR_Engine_oil_CH_BSIV_worksheet_name = MASsheetID.acell('B52').value                         # Get the worksheet name from cell B52
engine_oil_CH_BSIV_CH_MAS_ATR = ATR_Oil_Lub.worksheet(ATR_Engine_oil_CH_BSIV_worksheet_name)  # Open ATR CH engineoil worksheet

ATR_Engine_oil_CK_BSVI_worksheet_name = MASsheetID.acell('B53').value                         # Get the worksheet name from cell B52
engine_oil_CK_BSVI_CK_MAS_ATR = ATR_Oil_Lub.worksheet(ATR_Engine_oil_CK_BSVI_worksheet_name)  # Open ATR CK engineoil worksheet

#ATR_Report1_worksheet_name = MASsheetID.acell('B58').value                           #******This may not be required       # Get the worksheet name from report1, RR Km between and after last servicing
#ATR_Report1_Worksheet = ATR_Oil_Lub.worksheet(ATR_Report1_worksheet_name)              #This may not be required     # Open RR Km between and after last servicing worksheet

#STS FG
FG_Oil_Lub_Coolant_DEF_spreadsheet_id = MASsheetID.acell('C102').value                  # Get the spreadsheet ID from cell C51 ATR
FG_Oil_Lub = gc.open_by_key(FG_Oil_Lub_Coolant_DEF_spreadsheet_id)                    # Open ATR_Oil_Lub spreadsheet

FG_Engine_oil_CH_BSIV_worksheet_name = MASsheetID.acell('B102').value                         # Get the worksheet name from cell B2
engine_oil_CH_BSIV_CH_MAS_FG = FG_Oil_Lub.worksheet(FG_Engine_oil_CH_BSIV_worksheet_name)  # Open RR engineoil worksheet

FG_Engine_oil_CK_BSVI_worksheet_name = MASsheetID.acell('B103').value                         # Get the worksheet name from cell B52
engine_oil_CK_BSVI_CK_MAS_FG = FG_Oil_Lub.worksheet(FG_Engine_oil_CK_BSVI_worksheet_name)  # Open ATR CK engineoil worksheet

#FG_Report1_worksheet_name = MASsheetID.acell('B108').value                    #*****This may not be required            # Get the worksheet name from report1, RR Km between and after last servicing
#FG_Report1_Worksheet = FG_Oil_Lub.worksheet(FG_Report1_worksheet_name)          #This may not be required         # Open RR Km between and after last servicing worksheet



In [19]:
# Clear data in range A5:A200
clear_range_all_OdometersheetID = 'A5:R200'
clear_range_all_RR_Report1_Worksheet = ['B2:B200', 'AC2:AC200']

OdometersheetID.batch_clear([clear_range_all_OdometersheetID])
RR_Report1_Worksheet.batch_clear(clear_range_all_RR_Report1_Worksheet)

# === PROCESS BUS DATA ===

# Process SVP data
SVP_Odometer_id = OdometersheetID.acell('A2').value
Odometer_spreadsheet_SVP = gc.open_by_key(SVP_Odometer_id)
bus_number_SVP = sorted([worksheet.title for worksheet in Odometer_spreadsheet_SVP])
spreadsheet_name_SVP = Odometer_spreadsheet_SVP.title
start_row_svp = 2

# Process FG data
FG_Odometer_id = OdometersheetID.acell('B2').value
Odometer_spreadsheet_FG = gc.open_by_key(FG_Odometer_id)
bus_number_FG = sorted([worksheet.title for worksheet in Odometer_spreadsheet_FG])
spreadsheet_name_FG = Odometer_spreadsheet_FG.title
start_row_fg = start_row_svp + len(bus_number_SVP)

# Process Baratang data
BT_Odometer_id = OdometersheetID.acell('C2').value
Odometer_spreadsheet_BT = gc.open_by_key(BT_Odometer_id)
bus_number_BT = sorted([worksheet.title for worksheet in Odometer_spreadsheet_BT])
spreadsheet_name_BT = Odometer_spreadsheet_BT.title
start_row_bt = start_row_fg + len(bus_number_FG)

# Process Rangat data
RT_Odometer_id = OdometersheetID.acell('D2').value
Odometer_spreadsheet_RT = gc.open_by_key(RT_Odometer_id)
bus_number_RT = sorted([worksheet.title for worksheet in Odometer_spreadsheet_RT])
spreadsheet_name_RT = Odometer_spreadsheet_RT.title
start_row_rt = start_row_bt + len(bus_number_BT)

# Process Mayabunder data
MB_Odometer_id = OdometersheetID.acell('E2').value
Odometer_spreadsheet_MB = gc.open_by_key(MB_Odometer_id)
bus_number_MB = sorted([worksheet.title for worksheet in Odometer_spreadsheet_MB])
spreadsheet_name_MB = Odometer_spreadsheet_MB.title
start_row_mb = start_row_rt + len(bus_number_RT)

# Process Diglipur data
DP_Odometer_id = OdometersheetID.acell('F2').value
Odometer_spreadsheet_DP = gc.open_by_key(DP_Odometer_id)
bus_number_DP = sorted([worksheet.title for worksheet in Odometer_spreadsheet_DP])
spreadsheet_name_DP = Odometer_spreadsheet_DP.title
start_row_dp = start_row_mb + len(bus_number_MB)

# === UPDATE ODOMETERSHEERTID WITH BUS NUMBERS ===

# Function to update OdometersheetID column with bus numbers - one by one to avoid quote issues
def update_column_values(worksheet, column_letter, start_row, values_list):
    for i, value in enumerate(values_list):
        row = start_row + i
        cell = f"{column_letter}{row}"
        worksheet.update_acell(cell, value)

    print(f"Updated {len(values_list)} values in column {column_letter}")

# Function for batch update with string pre-processing to avoid quotes
def batch_update_with_preprocessing(worksheet, column_letter, start_row, values_list):
    # Create a list of dictionaries for batch update
    requests = []
    for i, value in enumerate(values_list):
        row = start_row + i
        # If value is numeric string, try to convert to number
        try:
            if value.isdigit():
                value = int(value)
        except:
            pass

        requests.append({
            'range': f'{column_letter}{row}',
            'values': [[value]]
        })

    if requests:
        worksheet.batch_update(requests)
        print(f"Batch updated {len(values_list)} values in column {column_letter}")

# Update spreadsheet names in row 4
OdometersheetID.update_acell('A4', spreadsheet_name_SVP)
OdometersheetID.update_acell('B4', spreadsheet_name_FG)
OdometersheetID.update_acell('C4', spreadsheet_name_BT)
OdometersheetID.update_acell('D4', spreadsheet_name_RT)
OdometersheetID.update_acell('E4', spreadsheet_name_MB)
OdometersheetID.update_acell('F4', spreadsheet_name_DP)
print("Updated all spreadsheet names in row 4")

# Use either individual updates or batch updates with preprocessing
USE_BATCH = True  # Set to False if you want individual cell updates

if USE_BATCH:
    # Update OdometersheetID columns in batches
    batch_update_with_preprocessing(OdometersheetID, 'A', 5, bus_number_SVP)
    batch_update_with_preprocessing(OdometersheetID, 'B', 5, bus_number_FG)
    batch_update_with_preprocessing(OdometersheetID, 'C', 5, bus_number_BT)
    batch_update_with_preprocessing(OdometersheetID, 'D', 5, bus_number_RT)
    batch_update_with_preprocessing(OdometersheetID, 'E', 5, bus_number_MB)
    batch_update_with_preprocessing(OdometersheetID, 'F', 5, bus_number_DP)
else:
    # Update OdometersheetID columns one by one
    update_column_values(OdometersheetID, 'A', 5, bus_number_SVP)
    update_column_values(OdometersheetID, 'B', 5, bus_number_FG)
    update_column_values(OdometersheetID, 'C', 5, bus_number_BT)
    update_column_values(OdometersheetID, 'D', 5, bus_number_RT)
    update_column_values(OdometersheetID, 'E', 5, bus_number_MB)
    update_column_values(OdometersheetID, 'F', 5, bus_number_DP)

# === UPDATE REPORT1 WORKSHEET WITH ALL BUS NUMBERS ===

# Combine all bus lists for Report1
all_buses_with_sources = []
for bus, source in zip(bus_number_SVP, [spreadsheet_name_SVP] * len(bus_number_SVP)):
    all_buses_with_sources.append((bus, source))
for bus, source in zip(bus_number_FG, [spreadsheet_name_FG] * len(bus_number_FG)):
    all_buses_with_sources.append((bus, source))
for bus, source in zip(bus_number_BT, [spreadsheet_name_BT] * len(bus_number_BT)):
    all_buses_with_sources.append((bus, source))
for bus, source in zip(bus_number_RT, [spreadsheet_name_RT] * len(bus_number_RT)):
    all_buses_with_sources.append((bus, source))
for bus, source in zip(bus_number_MB, [spreadsheet_name_MB] * len(bus_number_MB)):
    all_buses_with_sources.append((bus, source))
for bus, source in zip(bus_number_DP, [spreadsheet_name_DP] * len(bus_number_DP)):
    all_buses_with_sources.append((bus, source))

# Update Report1 with all bus data
if USE_BATCH:
    # Create batch update requests
    requests = []
    for i, (bus, source) in enumerate(all_buses_with_sources):
        row = i + 2  # Start from row 2
        # Try to convert numeric strings to integers
        try:
            if isinstance(bus, str) and bus.isdigit():
                bus = int(bus)
        except:
            pass

        requests.append({
            'range': f'B{row}',
            'values': [[bus]]
        })
        requests.append({
            'range': f'AC{row}',
            'values': [[source]]
        })

    if requests:
        RR_Report1_Worksheet.batch_update(requests)
        print(f"Batch updated {len(all_buses_with_sources)} bus entries in Report1 worksheet")
else:
    # Update one by one
    for i, (bus, source) in enumerate(all_buses_with_sources):
        row = i + 2  # Start from row 2
        RR_Report1_Worksheet.update_cell(row, 2, bus)  # Column B
        RR_Report1_Worksheet.update_cell(row, 29, source)  # Column AC

    print(f"Updated {len(all_buses_with_sources)} bus entries in Report1 worksheet")

print("All operations completed successfully!")

Updated all spreadsheet names in row 4
Batch updated 55 values in column A
Batch updated 20 values in column B
Batch updated 1 values in column C
Batch updated 1 values in column D
Batch updated 1 values in column E
Batch updated 1 values in column F
Batch updated 79 bus entries in Report1 worksheet
All operations completed successfully!


In [ ]:
#FG_Working

# Modified to process multiple engine oil worksheets, Added batch processing done

# Modified to process multiple engine oil worksheets with batch processing
# With fallback date display in M Column, For ATR_MAS_Engine Oil

# Make sure to include these imports at the top of your code
import re
from datetime import datetime
import random
import time
from collections import defaultdict
import gspread
import math
from concurrent.futures import ThreadPoolExecutor

# List of all worksheets to process
engine_oil_worksheets = [
    'engine_oil_CH_BSIV_CH_MAS_FG',
    'engine_oil_CK_BSVI_CK_MAS_FG'
]

# Function to parse custom date format 'DD,MMMyy' like '13,Apr24'
def parse_custom_date(date_str):
    if not date_str or date_str == '':
        return None

    pattern = r'(\d{1,2}),(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)(\d{2})'
    match = re.match(pattern, date_str)

    if match:
        day = int(match.group(1))
        month_str = match.group(2)
        year = int(match.group(3)) + 2000  # Assuming 20xx for the year

        month_map = {
            'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6,
            'Jul': 7, 'Aug': 8, 'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12
        }

        month = month_map[month_str]
        return datetime(year, month, day)

    return None

# Modified function to return both value and fallback date
def find_closest_date_value(sheet_data, target_date_str, value_column_index=12):
    # Parse all dates in the sheet
    dates = []
    parsed_dates = []
    values = []

    for row in sheet_data[1:]:  # Skip header
        try:
            if len(row) > value_column_index and len(row) > 1:
                date_str = row[1].strip()
                value = row[value_column_index]

                parsed_date = parse_custom_date(date_str)
                if parsed_date:
                    dates.append(date_str)
                    parsed_dates.append(parsed_date)
                    values.append(value)
        except:
            continue

    # Parse target date
    target_date = parse_custom_date(target_date_str)
    if not target_date:
        return None, None

    # First check for exact match
    for i, parsed_date in enumerate(parsed_dates):
        if parsed_date == target_date:
            return values[i], None  # Return value with no fallback date for exact matches

    # If no exact match, find closest earlier date
    closest_date = None
    closest_value = None
    max_date = None

    for i, parsed_date in enumerate(parsed_dates):
        if parsed_date < target_date and (max_date is None or parsed_date > max_date):
            max_date = parsed_date
            closest_date = dates[i]
            closest_value = values[i]

    return closest_value, closest_date  # Return both value and fallback date

# Function to process each worksheet
def process_worksheet(worksheet_name):
    print(f"\n🔄 Processing worksheet: {worksheet_name}")

    # Get the worksheet object
    worksheet = None
    try:
        # Assuming these are defined in your notebook environment
        worksheet = eval(worksheet_name)  # This will use the already defined variable in your notebook
    except:
        print(f"❌ Error: Worksheet '{worksheet_name}' not found or not accessible")
        return

    # Clear data in range L2:M, Odometer value in Engine Oil and Fallback date removal
    clear_range = 'L2:M'
    print(f"🧹 Clearing range {clear_range} in {worksheet_name}...")
    worksheet.batch_clear([clear_range])

    print("🔍 Starting data processing with date fallback search...")

    # 📥 Load main sheet
    print("📥 Loading worksheet data...")
    main_data = worksheet.get_all_values()
    print(f"✅ Loaded {len(main_data)-1} rows from worksheet")

    # Step 1: Collect unique (bus, date) and row mapping
    print("🔢 Mapping bus numbers and dates...")
    bus_date_map = defaultdict(set)
    row_map = {}  # maps row number → (bus, date)

    for i, row in enumerate(main_data[1:], start=2):
        try:
            date_str = row[1].strip()
            bus_num = row[4].strip()
            if date_str and bus_num:
                bus_date_map[bus_num].add(date_str)
                row_map[i] = (bus_num, date_str)
        except:
            continue

    print(f"✅ Found {len(bus_date_map)} unique bus numbers")
    print(f"✅ Found {len(row_map)} rows to process")

    # Create a cache for worksheet data to avoid redundant fetches
    bus_worksheet_cache = {}

    # Step 2: Fetch each bus worksheet once, collect all data and find exact or closest earlier date
    print("\n🔍 Searching for exact dates or closest earlier dates...")
    bus_date_value_map = {}
    match_type_map = {}  # Tracks whether each match is exact or fallback
    fallback_date_map = {}  # Store the actual fallback date used
    success_count = 0
    fallback_count = 0

    # Function to process a single bus
    def process_bus(bus_dates_tuple):
        bus, dates = bus_dates_tuple
        sheet_success = 0
        results = []

        try:
            # Check if we already have this bus's data in cache
            if bus in bus_worksheet_cache:
                sheet_data = bus_worksheet_cache[bus]
            else:
                # Fetch the bus worksheet data and store in cache
                sheet = Odometer_spreadsheet_FG.worksheet(bus)
                sheet_data = sheet.get_all_values()
                bus_worksheet_cache[bus] = sheet_data

            # Create a dictionary for faster exact date lookups
            date_value_lookup = {}
            for row in sheet_data[1:]:
                try:
                    if len(row) > 12:
                        date_str_key = row[1].strip()
                        date_value_lookup[date_str_key] = row[12]  # Column M
                except:
                    continue

            for date_str in dates:
                # Check for exact match using the lookup dictionary
                exact_value = date_value_lookup.get(date_str)

                if exact_value:
                    results.append((bus, date_str, exact_value, "exact", ""))
                    sheet_success += 1
                else:
                    # Try to find closest earlier date
                    value, fallback_date = find_closest_date_value(sheet_data, date_str, value_column_index=12)
                    if value:
                        results.append((bus, date_str, value, "fallback", fallback_date))
                        sheet_success += 1

            print(f"  ✅ Found {sheet_success}/{len(dates)} values for bus {bus}")
            return results

        except Exception as e:
            print(f"❌ Could not process sheet '{bus}': {str(e)}")
            return []

    # Process buses in parallel using ThreadPoolExecutor
    print(f"🚀 Processing {len(bus_date_map)} buses in parallel...")

    # Convert to list for ThreadPoolExecutor
    bus_dates_list = list(bus_date_map.items())

    # Use ThreadPoolExecutor for parallel processing
    # Adjust max_workers based on your system's capabilities
    with ThreadPoolExecutor(max_workers=5) as executor:
        all_results = list(executor.map(process_bus, bus_dates_list))

    # Process all results
    for bus_results in all_results:
        for bus, date_str, value, match_type, fallback_date in bus_results:
            bus_date_value_map[(bus, date_str)] = value
            match_type_map[(bus, date_str)] = match_type
            fallback_date_map[(bus, date_str)] = fallback_date

            if match_type == "exact":
                success_count += 1
            else:
                fallback_count += 1

    print(f"\n✅ Found values for {success_count} exact date matches")
    print(f"✅ Found values for {fallback_count} fallback date matches")
    print(f"❌ Could not find values for {len(row_map) - (success_count + fallback_count)} dates")

    # Step 3: Update values into Column L of main sheet and fallback dates into Column M
    print(f"\n📝 Preparing batch updates for {worksheet_name}...")

    # Prepare batch update for all cells at once
    batch_updates = []
    exact_match_cells = []
    fallback_match_cells = []

    for row_num, (bus, date_str) in row_map.items():
        value = bus_date_value_map.get((bus, date_str), "")
        fallback_date = fallback_date_map.get((bus, date_str), "")

        if value:
            # Add to batch updates list (row, col, value)
            batch_updates.append({
                'row': row_num,
                'col': 12,  # Column L
                'value': value
            })

            # Add fallback date update
            batch_updates.append({
                'row': row_num,
                'col': 13,  # Column M
                'value': fallback_date
            })

            # Track which cells need which formatting
            match_type = match_type_map.get((bus, date_str))
            if match_type == "exact":
                exact_match_cells.append(f"L{row_num}")
            elif match_type == "fallback":
                fallback_match_cells.append(f"L{row_num}")

    print(f"✅ Prepared {len(batch_updates)} cell updates")

    # Execute batch updates in chunks to avoid rate limits
    updated_count = 0
    chunk_size = 50  # Adjust based on API limits
    chunks = math.ceil(len(batch_updates) / chunk_size)

    print(f"🔄 Executing batch updates in {chunks} chunks...")

    for chunk_index in range(chunks):
        start_idx = chunk_index * chunk_size
        end_idx = min(start_idx + chunk_size, len(batch_updates))
        current_chunk = batch_updates[start_idx:end_idx]

        max_retries = 5
        retry_count = 0
        update_successful = False

        while not update_successful and retry_count < max_retries:
            try:
                # Prepare the batch update request
                cell_list = []

                for update in current_chunk:
                    cell = worksheet.cell(update['row'], update['col'])
                    cell.value = update['value']
                    cell_list.append(cell)

                # Execute the batch update
                worksheet.update_cells(cell_list, value_input_option='USER_ENTERED')

                update_successful = True
                updated_count += len(current_chunk) // 2  # Divide by 2 because we have 2 cells per row
                print(f"📊 Progress: Chunk {chunk_index+1}/{chunks} complete - {updated_count}/{len(batch_updates)//2} rows updated")

                # Add a small delay between chunks to avoid rate limits
                time.sleep(1)

            except Exception as e:
                retry_count += 1
                if "429" in str(e):
                    wait_time = (2 ** retry_count) + random.random()
                    print(f"⏳ Rate limit hit, waiting for {wait_time:.2f} seconds (retry {retry_count}/{max_retries})")
                    time.sleep(wait_time)
                else:
                    print(f"❌ Failed to update chunk {chunk_index+1}: {e}")
                    break

        if not update_successful:
            print(f"❌ Failed to update chunk {chunk_index+1} after {max_retries} retries")

    # Apply formatting in batches (currently commented out)
    print("\n🎨 Applying formatting to distinguish match types...")
    # You can uncomment and implement batch formatting logic if needed.

    print(f"\n✅ COMPLETE for {worksheet_name}: Updated {updated_count} rows in column L and column M")
    print(f"📊 Summary: {success_count} exact matches (normal format), {fallback_count} fallback matches (italic and right-aligned)")
    print(f"❌ {len(row_map) - updated_count} rows could not be updated")

# Main execution
print("🚀 Starting to process all engine oil worksheets...")

# Create a global cache for bus worksheets to reuse across all worksheets
global_bus_worksheet_cache = {}

# Process each worksheet in sequence
for worksheet_name in engine_oil_worksheets:
    print(f"\n{'='*80}\n📊 Processing worksheet: {worksheet_name}\n{'='*80}")
    process_worksheet(worksheet_name)
    # Add a delay between worksheets to avoid rate limiting
    time.sleep(2)

print("\n✅ All worksheets have been processed successfully!")
print("💡 Performance Summary:")
print(f"✓ Processed {len(engine_oil_worksheets)} worksheets with batch processing")
print(f"✓ Used parallel processing for bus data with ThreadPoolExecutor")
print(f"✓ Implemented caching to reduce API calls")
print(f"✓ Used batch updates to minimize API requests")


In [20]:
# Deepseek Revision ongoing in CODE #112 KM in Column D in Report1 Sheet, here it is searching all Odometer, i.e. from Odometer_spreadsheet_SVP or Odometer_spreadsheet_FG or Odometer_spreadsheet_BT or Odometer_spreadsheet_RT or Odometer_spreadsheet_MB or Odometer_spreadsheet_DP
# Working fine

from datetime import datetime
import re
import time
from collections import defaultdict

# CONFIGURATION
odometer_spreadsheets = [Odometer_spreadsheet_SVP, Odometer_spreadsheet_FG, Odometer_spreadsheet_BT,
                         Odometer_spreadsheet_RT, Odometer_spreadsheet_MB, Odometer_spreadsheet_DP]
spreadsheet_names = ["SVP", "FG", "BT", "RT", "MB", "DP"]

# Clear columns C and D
RR_Report1_Worksheet.batch_clear(["C2:D"])

# Get current date in expected format (e.g., "05,May25")
current_date_str = datetime.now().strftime('%d,%b%y')

# Get main data from Report1
main_data = RR_Report1_Worksheet.get_all_values()
header, rows = main_data[0], main_data[1:]

# Map each row to bus + date (using today's date)
row_map = {}
bus_set = set()
for i, row in enumerate(rows, start=2):
    try:
        bus = re.sub(r'[^a-zA-Z0-9]', '', row[1].strip()).upper()  # Clean bus name
        if bus:
            row_map[i] = (bus, current_date_str)
            bus_set.add(bus)
    except Exception as e:
        print(f"⚠️ Error processing row {i}: {e}")
        continue

print(f"🔎 Total buses to process: {len(bus_set)}")
print("Sample buses:", list(bus_set)[:5])  # Log first 5 buses

# Initialize lookup maps
bus_date_value_map = {}
match_type_map = {}
fallback_date_map = {}
source_spreadsheet_map = {}

# Function to find closest earlier date value
def find_closest_date_value(sheet_data, target_date_str, value_column_index=12):
    try:
        target_date = datetime.strptime(target_date_str, '%d,%b%y')
        valid_rows = []
        for row_idx, row in enumerate(sheet_data[1:], start=2):
            if len(row) <= value_column_index:
                continue
            date_cell = row[1].strip()
            if not date_cell:
                continue
            # Try multiple date formats
            try:
                row_date = datetime.strptime(date_cell, '%d,%b%y')
            except:
                try:
                    row_date = datetime.strptime(date_cell, '%d-%b-%y')
                except:
                    continue
            if row_date <= target_date and row[value_column_index].strip():
                valid_rows.append((row_date, date_cell, row[value_column_index].strip()))
        if valid_rows:
            valid_rows.sort(reverse=True)  # Newest date first
            best = valid_rows[0]
            return best[2], best[1]  # value, fallback_date
    except Exception as e:
        print(f"⚠️ Error in fallback for {target_date_str}: {e}")
    return None, ""

# Search logic with enhanced logging
for bus in bus_set:
    found = False
    for idx, spreadsheet in enumerate(odometer_spreadsheets):
        try:
            time.sleep(1)  # Avoid API throttling
            sheet = spreadsheet.worksheet(bus)
            data = sheet.get_all_values()
            print(f"\n✅ Found bus {bus} in {spreadsheet_names[idx]}")
            found = True
            # Check for exact date match
            exact_match = False
            for row in data[1:]:
                if len(row) > 12 and row[1].strip() == current_date_str and row[12].strip():
                    odometer_val = row[12].strip()
                    if odometer_val.isdigit():  # Validate numeric
                        bus_date_value_map[(bus, current_date_str)] = odometer_val
                        match_type_map[(bus, current_date_str)] = "exact"
                        fallback_date_map[(bus, current_date_str)] = ""
                        source_spreadsheet_map[(bus, current_date_str)] = spreadsheet_names[idx]
                        exact_match = True
                        print(f"📌 Exact match: {odometer_val} km on {current_date_str}")
                        break
            if not exact_match:
                # Fallback to closest date
                val, fallback_date = find_closest_date_value(data, current_date_str, 12)
                if val:
                    bus_date_value_map[(bus, current_date_str)] = val
                    match_type_map[(bus, current_date_str)] = "fallback"
                    fallback_date_map[(bus, current_date_str)] = fallback_date
                    source_spreadsheet_map[(bus, current_date_str)] = spreadsheet_names[idx]
                    print(f"🔄 Fallback match: {val} km on {fallback_date}")
            break  # Stop searching if found
        except Exception as e:
            if "not found" in str(e):
                print(f"❌ Bus {bus} not in {spreadsheet_names[idx]}")
                continue
            else:
                print(f"⚠️ Error accessing {bus} in {spreadsheet_names[idx]}: {e}")
    if not found:
        print(f"🚨 Bus {bus} not found in ANY spreadsheet!")

# Update Report1 with logging
print("\n📝 Writing data to Report1...")
updated_rows = []
for row_idx, (bus, date_str) in row_map.items():
    key = (bus, date_str)
    val = bus_date_value_map.get(key, "")
    fallback = fallback_date_map.get(key, "")
    match_type = match_type_map.get(key, "")
    source = source_spreadsheet_map.get(key, "")

    # Build updated values for Column C and D
    col_c = fallback if match_type == "fallback" else (source if match_type == "exact" else "")
    col_d = val if val else ""

    # Log updates for verification
    if not val:
        print(f"⚠️ No data for Bus {bus} | Date {date_str}")

    updated_rows.append([col_c, col_d])

# Batch update
if updated_rows:
    range_to_update = f"C2:D{len(updated_rows) + 1}"
    RR_Report1_Worksheet.update(range_to_update, updated_rows)
    print("✅ Report1 updated successfully!")
else:
    print("❌ No updates made to Report1.")

🔎 Total buses to process: 79
Sample buses: ['514', '604', '588', '593', '469']

✅ Found bus 514 in SVP
🔄 Fallback match: 181869 km on 06,May25

✅ Found bus 604 in SVP
🔄 Fallback match: 114004 km on 04,May25

✅ Found bus 588 in SVP
🔄 Fallback match: 285148 km on 05,May25
⚠️ Error accessing 593 in SVP: 593

✅ Found bus 593 in FG
🔄 Fallback match: 392975 km on 30,Apr25

✅ Found bus 469 in SVP
🔄 Fallback match: 236807 km on 29,Apr25

✅ Found bus 495 in SVP
🔄 Fallback match: 583802 km on 05,May25

✅ Found bus 476 in SVP
🔄 Fallback match: 387688 km on 06,May25

✅ Found bus 521 in SVP
🔄 Fallback match: 307305 km on 06,May25
⚠️ Error accessing 529 in SVP: 529

✅ Found bus 529 in FG
🔄 Fallback match: 360784 km on 18,Feb25
⚠️ Error accessing 592 in SVP: 592

✅ Found bus 592 in FG
🔄 Fallback match: 367136 km on 30,Apr25

✅ Found bus 563 in SVP
🔄 Fallback match: 731725 km on 30,Apr25

✅ Found bus 511 in SVP
🔄 Fallback match: 249770 km on 21,Apr25

✅ Found bus 442 in SVP
🔄 Fallback match: 276687 km

<ipython-input-20-b1d2be683ee7>:140: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  RR_Report1_Worksheet.update(range_to_update, updated_rows)


✅ Report1 updated successfully!


Key Modifications Needed
Multiple Target Worksheets:

engine_oil_CH_BSIV_CH_MAS_rr (existing)
engine_oil_CK_BSVI_CK_MAS_rr (new)
engine_oil_CK_BSVI_CK_MAS_ATR (new)
engine_oil_CH_BSIV_CH_MAS_ATR (new)

Multiple Source Spreadsheets:

Odometer_spreadsheet_SVP (existing)
Odometer_spreadsheet_FG (new)
Odometer_spreadsheet_BT (new)
Odometer_spreadsheet_RT (new)
Odometer_spreadsheet_MB (new)
Odometer_spreadsheet_DP (new)

In [ ]:
# Modified to process multiple engine oil worksheets SVP CH CK, ATR CH CK, Added batch processing done

# Modified to process multiple engine oil worksheets with batch processing
# With fallback date display in M Column, For ATR_MAS_Engine Oil

# Make sure to include these imports at the top of your code
import re
from datetime import datetime
import random
import time
from collections import defaultdict
import gspread
import math
from concurrent.futures import ThreadPoolExecutor

# List of all worksheets to process
engine_oil_worksheets = [
    'engine_oil_CH_BSIV_CH_MAS_ATR',
    'engine_oil_CK_BSVI_CK_MAS_ATR',
    'engine_oil_CH_BSIV_CH_MAS_rr',
    'engine_oil_CK_BSVI_CK_MAS_rr'
]

# Function to parse custom date format 'DD,MMMyy' like '13,Apr24'
def parse_custom_date(date_str):
    if not date_str or date_str == '':
        return None

    pattern = r'(\d{1,2}),(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)(\d{2})'
    match = re.match(pattern, date_str)

    if match:
        day = int(match.group(1))
        month_str = match.group(2)
        year = int(match.group(3)) + 2000  # Assuming 20xx for the year

        month_map = {
            'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6,
            'Jul': 7, 'Aug': 8, 'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12
        }

        month = month_map[month_str]
        return datetime(year, month, day)

    return None

# Modified function to return both value and fallback date
def find_closest_date_value(sheet_data, target_date_str, value_column_index=12):
    # Parse all dates in the sheet
    dates = []
    parsed_dates = []
    values = []

    for row in sheet_data[1:]:  # Skip header
        try:
            if len(row) > value_column_index and len(row) > 1:
                date_str = row[1].strip()
                value = row[value_column_index]

                parsed_date = parse_custom_date(date_str)
                if parsed_date:
                    dates.append(date_str)
                    parsed_dates.append(parsed_date)
                    values.append(value)
        except:
            continue

    # Parse target date
    target_date = parse_custom_date(target_date_str)
    if not target_date:
        return None, None

    # First check for exact match
    for i, parsed_date in enumerate(parsed_dates):
        if parsed_date == target_date:
            return values[i], None  # Return value with no fallback date for exact matches

    # If no exact match, find closest earlier date
    closest_date = None
    closest_value = None
    max_date = None

    for i, parsed_date in enumerate(parsed_dates):
        if parsed_date < target_date and (max_date is None or parsed_date > max_date):
            max_date = parsed_date
            closest_date = dates[i]
            closest_value = values[i]

    return closest_value, closest_date  # Return both value and fallback date

# Function to process each worksheet
def process_worksheet(worksheet_name):
    print(f"\n🔄 Processing worksheet: {worksheet_name}")

    # Get the worksheet object
    worksheet = None
    try:
        # Assuming these are defined in your notebook environment
        worksheet = eval(worksheet_name)  # This will use the already defined variable in your notebook
    except:
        print(f"❌ Error: Worksheet '{worksheet_name}' not found or not accessible")
        return

    # Clear data in range L2:M, Odometer value in Engine Oil and Fallback date removal
    clear_range = 'L2:M'
    print(f"🧹 Clearing range {clear_range} in {worksheet_name}...")
    worksheet.batch_clear([clear_range])

    print("🔍 Starting data processing with date fallback search...")

    # 📥 Load main sheet
    print("📥 Loading worksheet data...")
    main_data = worksheet.get_all_values()
    print(f"✅ Loaded {len(main_data)-1} rows from worksheet")

    # Step 1: Collect unique (bus, date) and row mapping
    print("🔢 Mapping bus numbers and dates...")
    bus_date_map = defaultdict(set)
    row_map = {}  # maps row number → (bus, date)

    for i, row in enumerate(main_data[1:], start=2):
        try:
            date_str = row[1].strip()
            bus_num = row[4].strip()
            if date_str and bus_num:
                bus_date_map[bus_num].add(date_str)
                row_map[i] = (bus_num, date_str)
        except:
            continue

    print(f"✅ Found {len(bus_date_map)} unique bus numbers")
    print(f"✅ Found {len(row_map)} rows to process")

    # Create a cache for worksheet data to avoid redundant fetches
    bus_worksheet_cache = {}

    # Step 2: Fetch each bus worksheet once, collect all data and find exact or closest earlier date
    print("\n🔍 Searching for exact dates or closest earlier dates...")
    bus_date_value_map = {}
    match_type_map = {}  # Tracks whether each match is exact or fallback
    fallback_date_map = {}  # Store the actual fallback date used
    success_count = 0
    fallback_count = 0

    # Function to process a single bus
    def process_bus(bus_dates_tuple):
        bus, dates = bus_dates_tuple
        sheet_success = 0
        results = []

        try:
            # Check if we already have this bus's data in cache
            if bus in bus_worksheet_cache:
                sheet_data = bus_worksheet_cache[bus]
            else:
                # Fetch the bus worksheet data and store in cache
                sheet = Odometer_spreadsheet_SVP.worksheet(bus)
                sheet_data = sheet.get_all_values()
                bus_worksheet_cache[bus] = sheet_data

            # Create a dictionary for faster exact date lookups
            date_value_lookup = {}
            for row in sheet_data[1:]:
                try:
                    if len(row) > 12:
                        date_str_key = row[1].strip()
                        date_value_lookup[date_str_key] = row[12]  # Column M
                except:
                    continue

            for date_str in dates:
                # Check for exact match using the lookup dictionary
                exact_value = date_value_lookup.get(date_str)

                if exact_value:
                    results.append((bus, date_str, exact_value, "exact", ""))
                    sheet_success += 1
                else:
                    # Try to find closest earlier date
                    value, fallback_date = find_closest_date_value(sheet_data, date_str, value_column_index=12)
                    if value:
                        results.append((bus, date_str, value, "fallback", fallback_date))
                        sheet_success += 1

            print(f"  ✅ Found {sheet_success}/{len(dates)} values for bus {bus}")
            return results

        except Exception as e:
            print(f"❌ Could not process sheet '{bus}': {str(e)}")
            return []

    # Process buses in parallel using ThreadPoolExecutor
    print(f"🚀 Processing {len(bus_date_map)} buses in parallel...")

    # Convert to list for ThreadPoolExecutor
    bus_dates_list = list(bus_date_map.items())

    # Use ThreadPoolExecutor for parallel processing
    # Adjust max_workers based on your system's capabilities
    with ThreadPoolExecutor(max_workers=5) as executor:
        all_results = list(executor.map(process_bus, bus_dates_list))

    # Process all results
    for bus_results in all_results:
        for bus, date_str, value, match_type, fallback_date in bus_results:
            bus_date_value_map[(bus, date_str)] = value
            match_type_map[(bus, date_str)] = match_type
            fallback_date_map[(bus, date_str)] = fallback_date

            if match_type == "exact":
                success_count += 1
            else:
                fallback_count += 1

    print(f"\n✅ Found values for {success_count} exact date matches")
    print(f"✅ Found values for {fallback_count} fallback date matches")
    print(f"❌ Could not find values for {len(row_map) - (success_count + fallback_count)} dates")

    # Step 3: Update values into Column L of main sheet and fallback dates into Column M
    print(f"\n📝 Preparing batch updates for {worksheet_name}...")

    # Prepare batch update for all cells at once
    batch_updates = []
    exact_match_cells = []
    fallback_match_cells = []

    for row_num, (bus, date_str) in row_map.items():
        value = bus_date_value_map.get((bus, date_str), "")
        fallback_date = fallback_date_map.get((bus, date_str), "")

        if value:
            # Add to batch updates list (row, col, value)
            batch_updates.append({
                'row': row_num,
                'col': 12,  # Column L
                'value': value
            })

            # Add fallback date update
            batch_updates.append({
                'row': row_num,
                'col': 13,  # Column M
                'value': fallback_date
            })

            # Track which cells need which formatting
            match_type = match_type_map.get((bus, date_str))
            if match_type == "exact":
                exact_match_cells.append(f"L{row_num}")
            elif match_type == "fallback":
                fallback_match_cells.append(f"L{row_num}")

    print(f"✅ Prepared {len(batch_updates)} cell updates")

    # Execute batch updates in chunks to avoid rate limits
    updated_count = 0
    chunk_size = 50  # Adjust based on API limits
    chunks = math.ceil(len(batch_updates) / chunk_size)

    print(f"🔄 Executing batch updates in {chunks} chunks...")

    for chunk_index in range(chunks):
        start_idx = chunk_index * chunk_size
        end_idx = min(start_idx + chunk_size, len(batch_updates))
        current_chunk = batch_updates[start_idx:end_idx]

        max_retries = 5
        retry_count = 0
        update_successful = False

        while not update_successful and retry_count < max_retries:
            try:
                # Prepare the batch update request
                cell_list = []

                for update in current_chunk:
                    cell = worksheet.cell(update['row'], update['col'])
                    cell.value = update['value']
                    cell_list.append(cell)

                # Execute the batch update
                worksheet.update_cells(cell_list, value_input_option='USER_ENTERED')

                update_successful = True
                updated_count += len(current_chunk) // 2  # Divide by 2 because we have 2 cells per row
                print(f"📊 Progress: Chunk {chunk_index+1}/{chunks} complete - {updated_count}/{len(batch_updates)//2} rows updated")

                # Add a small delay between chunks to avoid rate limits
                time.sleep(1)

            except Exception as e:
                retry_count += 1
                if "429" in str(e):
                    wait_time = (2 ** retry_count) + random.random()
                    print(f"⏳ Rate limit hit, waiting for {wait_time:.2f} seconds (retry {retry_count}/{max_retries})")
                    time.sleep(wait_time)
                else:
                    print(f"❌ Failed to update chunk {chunk_index+1}: {e}")
                    break

        if not update_successful:
            print(f"❌ Failed to update chunk {chunk_index+1} after {max_retries} retries")

    # Apply formatting in batches (currently commented out)
    print("\n🎨 Applying formatting to distinguish match types...")
    # You can uncomment and implement batch formatting logic if needed.

    print(f"\n✅ COMPLETE for {worksheet_name}: Updated {updated_count} rows in column L and column M")
    print(f"📊 Summary: {success_count} exact matches (normal format), {fallback_count} fallback matches (italic and right-aligned)")
    print(f"❌ {len(row_map) - updated_count} rows could not be updated")

# Main execution
print("🚀 Starting to process all engine oil worksheets...")

# Create a global cache for bus worksheets to reuse across all worksheets
global_bus_worksheet_cache = {}

# Process each worksheet in sequence
for worksheet_name in engine_oil_worksheets:
    print(f"\n{'='*80}\n📊 Processing worksheet: {worksheet_name}\n{'='*80}")
    process_worksheet(worksheet_name)
    # Add a delay between worksheets to avoid rate limiting
    time.sleep(2)

print("\n✅ All worksheets have been processed successfully!")
print("💡 Performance Summary:")
print(f"✓ Processed {len(engine_oil_worksheets)} worksheets with batch processing")
print(f"✓ Used parallel processing for bus data with ThreadPoolExecutor")
print(f"✓ Implemented caching to reduce API calls")
print(f"✓ Used batch updates to minimize API requests")


In [ ]:
# Lean version - Only processes rows with empty L column
# With fallback date display in M Column, For SVP and RR and ATR, CH and CK MAS

import re
from datetime import datetime
import random
import time
from collections import defaultdict
import gspread
import math
from concurrent.futures import ThreadPoolExecutor

# List of all worksheets to process
engine_oil_worksheets = [
    'engine_oil_CH_BSIV_CH_MAS_ATR',
    'engine_oil_CK_BSVI_CK_MAS_ATR',
    'engine_oil_CH_BSIV_CH_MAS_rr',
    'engine_oil_CK_BSVI_CK_MAS_rr'
]

# Function to parse custom date format 'DD,MMMyy' like '13,Apr24'
def parse_custom_date(date_str):
    if not date_str or date_str == '':
        return None

    pattern = r'(\d{1,2}),(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)(\d{2})'
    match = re.match(pattern, date_str)

    if match:
        day = int(match.group(1))
        month_str = match.group(2)
        year = int(match.group(3)) + 2000  # Assuming 20xx for the year

        month_map = {
            'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6,
            'Jul': 7, 'Aug': 8, 'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12
        }

        month = month_map[month_str]
        return datetime(year, month, day)

    return None

# Modified function to return both value and fallback date
def find_closest_date_value(sheet_data, target_date_str, value_column_index=12):
    # Parse all dates in the sheet
    dates = []
    parsed_dates = []
    values = []

    for row in sheet_data[1:]:  # Skip header
        try:
            if len(row) > value_column_index and len(row) > 1:
                date_str = row[1].strip()
                value = row[value_column_index]

                parsed_date = parse_custom_date(date_str)
                if parsed_date:
                    dates.append(date_str)
                    parsed_dates.append(parsed_date)
                    values.append(value)
        except:
            continue

    # Parse target date
    target_date = parse_custom_date(target_date_str)
    if not target_date:
        return None, None

    # First check for exact match
    for i, parsed_date in enumerate(parsed_dates):
        if parsed_date == target_date:
            return values[i], None  # Return value with no fallback date for exact matches

    # If no exact match, find closest earlier date
    closest_date = None
    closest_value = None
    max_date = None

    for i, parsed_date in enumerate(parsed_dates):
        if parsed_date < target_date and (max_date is None or parsed_date > max_date):
            max_date = parsed_date
            closest_date = dates[i]
            closest_value = values[i]

    return closest_value, closest_date  # Return both value and fallback date

# Global cache for bus worksheets to reuse across all worksheets
global_bus_worksheet_cache = {}

# Function to process each worksheet - LEAN VERSION (only empty L cells)
def process_worksheet_lean(worksheet_name):
    print(f"\n🔄 Processing worksheet: {worksheet_name}")

    # Get the worksheet object
    worksheet = None
    try:
        # Assuming these are defined in your notebook environment
        worksheet = eval(worksheet_name)  # This will use the already defined variable in your notebook
    except:
        print(f"❌ Error: Worksheet '{worksheet_name}' not found or not accessible")
        return

    # NO CLEARING OF RANGES - we're only updating empty cells
    print("🔍 Starting data processing for EMPTY L cells only...")

    # 📥 Load main sheet
    print("📥 Loading worksheet data...")
    main_data = worksheet.get_all_values()
    print(f"✅ Loaded {len(main_data)-1} rows from worksheet")

    # Step 1: Collect unique (bus, date) and row mapping ONLY FOR EMPTY L CELLS
    print("🔢 Mapping bus numbers and dates for empty L cells...")
    bus_date_map = defaultdict(set)
    row_map = {}  # maps row number → (bus, date)
    empty_cell_count = 0

    for i, row in enumerate(main_data[1:], start=2):
        try:
            # Check if L column (index 11) is empty
            l_value = row[11].strip() if len(row) > 11 else ""

            if not l_value:  # Only process if L column is empty
                date_str = row[1].strip()
                bus_num = row[4].strip()
                if date_str and bus_num:
                    bus_date_map[bus_num].add(date_str)
                    row_map[i] = (bus_num, date_str)
                    empty_cell_count += 1
        except:
            continue

    print(f"✅ Found {len(bus_date_map)} unique bus numbers with empty L cells")
    print(f"✅ Found {empty_cell_count} empty L cells to process")

    # Exit early if no empty cells to process
    if empty_cell_count == 0:
        print(f"✓ No empty L cells to process in {worksheet_name}. Moving to next worksheet.")
        return

    # Step 2: Fetch each bus worksheet once, collect all data and find exact or closest earlier date
    print("\n🔍 Searching for exact dates or closest earlier dates...")
    bus_date_value_map = {}
    match_type_map = {}  # Tracks whether each match is exact or fallback
    fallback_date_map = {}  # Store the actual fallback date used
    success_count = 0
    fallback_count = 0

    # Function to process a single bus
    def process_bus(bus_dates_tuple):
        bus, dates = bus_dates_tuple
        sheet_success = 0
        results = []

        try:
            # Check if we already have this bus's data in cache
            if bus in global_bus_worksheet_cache:
                sheet_data = global_bus_worksheet_cache[bus]
            else:
                # Fetch the bus worksheet data and store in cache
                sheet = Odometer_spreadsheet_SVP.worksheet(bus)
                sheet_data = sheet.get_all_values()
                global_bus_worksheet_cache[bus] = sheet_data

            # Create a dictionary for faster exact date lookups
            date_value_lookup = {}
            for row in sheet_data[1:]:
                try:
                    if len(row) > 12:
                        date_str_key = row[1].strip()
                        date_value_lookup[date_str_key] = row[12]  # Column M
                except:
                    continue

            for date_str in dates:
                # Check for exact match using the lookup dictionary
                exact_value = date_value_lookup.get(date_str)

                if exact_value:
                    results.append((bus, date_str, exact_value, "exact", ""))
                    sheet_success += 1
                else:
                    # Try to find closest earlier date
                    value, fallback_date = find_closest_date_value(sheet_data, date_str, value_column_index=12)
                    if value:
                        results.append((bus, date_str, value, "fallback", fallback_date))
                        sheet_success += 1

            print(f"  ✅ Found {sheet_success}/{len(dates)} values for bus {bus}")
            return results

        except Exception as e:
            print(f"❌ Could not process sheet '{bus}': {str(e)}")
            return []

    # Only process if we have buses to check
    if len(bus_date_map) > 0:
        # Convert to list for ThreadPoolExecutor
        bus_dates_list = list(bus_date_map.items())

        # Use ThreadPoolExecutor for parallel processing
        max_workers = min(5, len(bus_dates_list))  # Adjust based on data size
        with ThreadPoolExecutor(max_workers=max_workers) as executor:
            all_results = list(executor.map(process_bus, bus_dates_list))

        # Process all results
        for bus_results in all_results:
            for bus, date_str, value, match_type, fallback_date in bus_results:
                bus_date_value_map[(bus, date_str)] = value
                match_type_map[(bus, date_str)] = match_type
                fallback_date_map[(bus, date_str)] = fallback_date

                if match_type == "exact":
                    success_count += 1
                else:
                    fallback_count += 1
    else:
        print("⚠️ No buses to process - all L cells have values already")

    print(f"\n✅ Found values for {success_count} exact date matches")
    print(f"✅ Found values for {fallback_count} fallback date matches")
    print(f"❌ Could not find values for {len(row_map) - (success_count + fallback_count)} dates")

    # Step 3: Update values into Column L of main sheet and fallback dates into Column M
    print(f"\n📝 Preparing batch updates for {worksheet_name}...")

    # Prepare batch update for all cells at once
    batch_updates = []
    exact_match_cells = []
    fallback_match_cells = []

    for row_num, (bus, date_str) in row_map.items():
        value = bus_date_value_map.get((bus, date_str), "")
        fallback_date = fallback_date_map.get((bus, date_str), "")

        if value:
            # Add to batch updates list (row, col, value)
            batch_updates.append({
                'row': row_num,
                'col': 12,  # Column L
                'value': value
            })

            # Add fallback date update
            batch_updates.append({
                'row': row_num,
                'col': 13,  # Column M
                'value': fallback_date
            })

            # Track which cells need which formatting
            match_type = match_type_map.get((bus, date_str))
            if match_type == "exact":
                exact_match_cells.append(f"L{row_num}")
            elif match_type == "fallback":
                fallback_match_cells.append(f"L{row_num}")

    print(f"✅ Prepared {len(batch_updates)} cell updates")

    # Skip if no updates needed
    if len(batch_updates) == 0:
        print("✓ No updates needed for this worksheet")
        return

    # Execute batch updates in chunks to avoid rate limits
    updated_count = 0
    chunk_size = 50  # Adjust based on API limits
    chunks = math.ceil(len(batch_updates) / chunk_size)

    print(f"🔄 Executing batch updates in {chunks} chunks...")

    for chunk_index in range(chunks):
        start_idx = chunk_index * chunk_size
        end_idx = min(start_idx + chunk_size, len(batch_updates))
        current_chunk = batch_updates[start_idx:end_idx]

        max_retries = 5
        retry_count = 0
        update_successful = False

        while not update_successful and retry_count < max_retries:
            try:
                # Prepare the batch update request
                cell_list = []

                for update in current_chunk:
                    cell = worksheet.cell(update['row'], update['col'])
                    cell.value = update['value']
                    cell_list.append(cell)

                # Execute the batch update
                worksheet.update_cells(cell_list, value_input_option='USER_ENTERED')

                update_successful = True
                updated_count += len(current_chunk) // 2  # Divide by 2 because we have 2 cells per row
                print(f"📊 Progress: Chunk {chunk_index+1}/{chunks} complete - {updated_count}/{len(batch_updates)//2} rows updated")

                # Add a small delay between chunks to avoid rate limits
                time.sleep(1)

            except Exception as e:
                retry_count += 1
                if "429" in str(e):
                    wait_time = (2 ** retry_count) + random.random()
                    print(f"⏳ Rate limit hit, waiting for {wait_time:.2f} seconds (retry {retry_count}/{max_retries})")
                    time.sleep(wait_time)
                else:
                    print(f"❌ Failed to update chunk {chunk_index+1}: {e}")
                    break

        if not update_successful:
            print(f"❌ Failed to update chunk {chunk_index+1} after {max_retries} retries")

    # Apply formatting in batches (currently commented out)
    print("\n🎨 Applying formatting to distinguish match types...")
    # You can uncomment and implement batch formatting logic if needed.

    print(f"\n✅ COMPLETE for {worksheet_name}: Updated {updated_count} rows in column L and column M")
    print(f"📊 Summary: {success_count} exact matches (normal format), {fallback_count} fallback matches (italic and right-aligned)")
    print(f"❌ {len(row_map) - updated_count*2} rows could not be updated")

# Main execution - LEAN VERSION
print("🚀 Starting LEAN processing - only empty L cells...")

# Process each worksheet in sequence
for worksheet_name in engine_oil_worksheets:
    print(f"\n{'='*80}\n📊 Processing worksheet: {worksheet_name}\n{'='*80}")
    process_worksheet_lean(worksheet_name)
    # Add a delay between worksheets to avoid rate limiting
    time.sleep(2)

print("\n✅ All worksheets have been processed successfully!")
print("💡 Performance Summary:")
print(f"✓ Processed {len(engine_oil_worksheets)} worksheets with LEAN mode (empty L cells only)")
print(f"✓ Used parallel processing for bus data with ThreadPoolExecutor")
print(f"✓ Implemented caching to reduce API calls")
print(f"✓ Used batch updates to minimize API requests")

Below codes are redundent, No need to run, Use this If required in future

In [21]:
# With fallback date display in M Column, For RR_MAS_Engine OIl_ Best Code

# Make sure to include these imports at the top of your code
import re
from datetime import datetime
import random
import time
from collections import defaultdict
import gspread


# Clear data in range L2:M, Odometer value in Engine Oil and Fallback date removal
clear_range_all_engine_oil_CH_BSIV_CH_MAS_rr = 'L2:M'
engine_oil_CH_BSIV_CH_MAS_rr.batch_clear([clear_range_all_engine_oil_CH_BSIV_CH_MAS_rr])



# Function to parse custom date format 'DD,MMMyy' like '13,Apr24'
def parse_custom_date(date_str):
    if not date_str or date_str == '':
        return None

    pattern = r'(\d{1,2}),(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)(\d{2})'
    match = re.match(pattern, date_str)

    if match:
        day = int(match.group(1))
        month_str = match.group(2)
        year = int(match.group(3)) + 2000  # Assuming 20xx for the year

        month_map = {
            'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6,
            'Jul': 7, 'Aug': 8, 'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12
        }

        month = month_map[month_str]
        return datetime(year, month, day)

    return None

# Modified function to return both value and fallback date
def find_closest_date_value(sheet_data, target_date_str, value_column_index=12):
    # Parse all dates in the sheet
    dates = []
    parsed_dates = []
    values = []

    for row in sheet_data[1:]:  # Skip header
        try:
            if len(row) > value_column_index and len(row) > 1:
                date_str = row[1].strip()
                value = row[value_column_index]

                parsed_date = parse_custom_date(date_str)
                if parsed_date:
                    dates.append(date_str)
                    parsed_dates.append(parsed_date)
                    values.append(value)
        except:
            continue

    # Parse target date
    target_date = parse_custom_date(target_date_str)
    if not target_date:
        return None, None

    # First check for exact match
    for i, parsed_date in enumerate(parsed_dates):
        if parsed_date == target_date:
            return values[i], None  # Return value with no fallback date for exact matches

    # If no exact match, find closest earlier date
    closest_date = None
    closest_value = None
    max_date = None

    for i, parsed_date in enumerate(parsed_dates):
        if parsed_date < target_date and (max_date is None or parsed_date > max_date):
            max_date = parsed_date
            closest_date = dates[i]
            closest_value = values[i]

    return closest_value, closest_date  # Return both value and fallback date

print("🔍 Starting data processing with date fallback search...")

# 📥 Load main sheet
print("📥 Loading main sheet data...")
main_data = engine_oil_CH_BSIV_CH_MAS_rr.get_all_values()
print(f"✅ Loaded {len(main_data)-1} rows from main sheet")

# Step 1: Collect unique (bus, date) and row mapping
print("🔢 Mapping bus numbers and dates...")
bus_date_map = defaultdict(set)
row_map = {}  # maps row number → (bus, date)

for i, row in enumerate(main_data[1:], start=2):
    try:
        date_str = row[1].strip()
        bus_num = row[4].strip()
        if date_str and bus_num:
            bus_date_map[bus_num].add(date_str)
            row_map[i] = (bus_num, date_str)
    except:
        continue

print(f"✅ Found {len(bus_date_map)} unique bus numbers")
print(f"✅ Found {len(row_map)} rows to process")

# Step 2: Fetch each bus worksheet once, collect all data and find exact or closest earlier date
print("\n🔍 Searching for exact dates or closest earlier dates...")
bus_date_value_map = {}
match_type_map = {}  # Tracks whether each match is exact or fallback
fallback_date_map = {}  # Store the actual fallback date used
success_count = 0
fallback_count = 0

for bus, dates in bus_date_map.items():
    print(f"📊 Processing bus {bus} ({len(dates)} dates)...")
    try:
        sheet = Odometer_spreadsheet_SVP.worksheet(bus)
        sheet_data = sheet.get_all_values()
        sheet_success = 0

        for date_str in dates:
            # Find exact match first
            exact_value = None
            for row in sheet_data[1:]:
                try:
                    if len(row) > 12 and row[1].strip() == date_str:
                        exact_value = row[12]  # Column M
                        break
                except:
                    continue

            if exact_value:
                bus_date_value_map[(bus, date_str)] = exact_value
                match_type_map[(bus, date_str)] = "exact"
                fallback_date_map[(bus, date_str)] = ""  # No fallback date for exact matches
                sheet_success += 1
                success_count += 1
            else:
                # Try to find closest earlier date
                value, fallback_date = find_closest_date_value(sheet_data, date_str, value_column_index=12)
                if value:
                    bus_date_value_map[(bus, date_str)] = value
                    match_type_map[(bus, date_str)] = "fallback"
                    fallback_date_map[(bus, date_str)] = fallback_date  # Store the fallback date used
                    sheet_success += 1
                    fallback_count += 1

        print(f"  ✅ Found {sheet_success}/{len(dates)} values for bus {bus}")

    except Exception as e:
        print(f"❌ Could not process sheet '{bus}': {str(e)}")
        continue

print(f"\n✅ Found values for {success_count} exact date matches")
print(f"✅ Found values for {fallback_count} fallback date matches")
print(f"❌ Could not find values for {len(row_map) - (success_count + fallback_count)} dates")

# Step 3: Update values into Column L of main sheet and fallback dates into Column M
print("\n📝 Updating values and formatting in main sheet...")
updated_count = 0

# Get the worksheet object for batch updates
worksheet = engine_oil_CH_BSIV_CH_MAS_rr

# Prepare batch formatting requests for different match types
exact_match_cells = []
fallback_match_cells = []

for row_num, (bus, date_str) in row_map.items():
    value = bus_date_value_map.get((bus, date_str), "")
    fallback_date = fallback_date_map.get((bus, date_str), "")

    if value:
        max_retries = 5
        retry_count = 0
        update_successful = False

        while not update_successful and retry_count < max_retries:
            try:
                # Update cell value in column L (value)
                worksheet.update_cell(row_num, 12, value)  # Column L = col 12

                # Update column M with fallback date if applicable
                worksheet.update_cell(row_num, 13, fallback_date)  # Column M = col 13

                # Track which cells need which formatting
                match_type = match_type_map.get((bus, date_str))
                if match_type == "exact":
                    # For exact matches - normal formatting
                    exact_match_cells.append(f"L{row_num}")
                elif match_type == "fallback":
                    # For fallback matches - italic and right-aligned
                    fallback_match_cells.append(f"L{row_num}")

                update_successful = True
                updated_count += 1
                if updated_count % 10 == 0:  # Progress update every 10 rows
                    print(f"📊 Progress: Updated {updated_count}/{len(bus_date_value_map)} values")
            except Exception as e:
                retry_count += 1
                if "429" in str(e):
                    # Calculate wait time with exponential backoff
                    wait_time = (2 ** retry_count) + random.random()
                    print(f"⏳ Rate limit hit, waiting for {wait_time:.2f} seconds (retry {retry_count}/{max_retries})")
                    time.sleep(wait_time)
                else:
                    print(f"❌ Failed to update row {row_num}: {e}")
                    break

        if not update_successful:
            print(f"❌ Failed to update row {row_num} after {max_retries} retries")

# Apply formatting in batches to avoid rate limiting
print("\n🎨 Applying formatting to distinguish match types...")

print(f"\n✅ COMPLETE: Updated {updated_count} rows in column L and column M")
print(f"📊 Summary: {success_count} exact matches (normal format), {fallback_count} fallback matches (italic and right-aligned)")
print(f"❌ {len(row_map) - updated_count} rows could not be updated")

🔍 Starting data processing with date fallback search...
📥 Loading main sheet data...
✅ Loaded 932 rows from main sheet
🔢 Mapping bus numbers and dates...
✅ Found 76 unique bus numbers
✅ Found 917 rows to process

🔍 Searching for exact dates or closest earlier dates...
📊 Processing bus 504 (17 dates)...
  ✅ Found 17/17 values for bus 504
📊 Processing bus 326 (5 dates)...
❌ Could not process sheet '326': 326
📊 Processing bus 477 (3 dates)...
❌ Could not process sheet '477': 477
📊 Processing bus 322 (8 dates)...
❌ Could not process sheet '322': 322
📊 Processing bus 317 (55 dates)...
❌ Could not process sheet '317': 317
📊 Processing bus 486 (15 dates)...
  ✅ Found 15/15 values for bus 486
📊 Processing bus 342 (15 dates)...
❌ Could not process sheet '342': 342
📊 Processing bus 585 (39 dates)...
  ✅ Found 39/39 values for bus 585
📊 Processing bus 493 (21 dates)...
❌ Could not process sheet '493': 493
📊 Processing bus 460 (28 dates)...
  ✅ Found 28/28 values for bus 460
📊 Processing bus 381 (

In [ ]:
# Modified to process multiple engine oil worksheets SVP CH CK, ATR CH CK, No batch processing done

# Make sure to include these imports at the top of your code
import re
from datetime import datetime
import random
import time
from collections import defaultdict
import gspread

# List of all worksheets to process
engine_oil_worksheets = [
    'engine_oil_CH_BSIV_CH_MAS_ATR',
    'engine_oil_CK_BSVI_CK_MAS_ATR',
    'engine_oil_CH_BSIV_CH_MAS_rr',
    'engine_oil_CK_BSVI_CK_MAS_rr'
]

# Function to parse custom date format 'DD,MMMyy' like '13,Apr24'
def parse_custom_date(date_str):
    if not date_str or date_str == '':
        return None

    pattern = r'(\d{1,2}),(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)(\d{2})'
    match = re.match(pattern, date_str)

    if match:
        day = int(match.group(1))
        month_str = match.group(2)
        year = int(match.group(3)) + 2000  # Assuming 20xx for the year

        month_map = {
            'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6,
            'Jul': 7, 'Aug': 8, 'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12
        }

        month = month_map[month_str]
        return datetime(year, month, day)

    return None

# Modified function to return both value and fallback date
def find_closest_date_value(sheet_data, target_date_str, value_column_index=12):
    # Parse all dates in the sheet
    dates = []
    parsed_dates = []
    values = []

    for row in sheet_data[1:]:  # Skip header
        try:
            if len(row) > value_column_index and len(row) > 1:
                date_str = row[1].strip()
                value = row[value_column_index]

                parsed_date = parse_custom_date(date_str)
                if parsed_date:
                    dates.append(date_str)
                    parsed_dates.append(parsed_date)
                    values.append(value)
        except:
            continue

    # Parse target date
    target_date = parse_custom_date(target_date_str)
    if not target_date:
        return None, None

    # First check for exact match
    for i, parsed_date in enumerate(parsed_dates):
        if parsed_date == target_date:
            return values[i], None  # Return value with no fallback date for exact matches

    # If no exact match, find closest earlier date
    closest_date = None
    closest_value = None
    max_date = None

    for i, parsed_date in enumerate(parsed_dates):
        if parsed_date < target_date and (max_date is None or parsed_date > max_date):
            max_date = parsed_date
            closest_date = dates[i]
            closest_value = values[i]

    return closest_value, closest_date  # Return both value and fallback date

# Function to process each worksheet
def process_worksheet(worksheet_name):
    print(f"\n🔄 Processing worksheet: {worksheet_name}")

    # Get the worksheet object
    worksheet = None
    try:
        # Assuming these are defined in your notebook environment
        worksheet = eval(worksheet_name)  # This will use the already defined variable in your notebook
    except:
        print(f"❌ Error: Worksheet '{worksheet_name}' not found or not accessible")
        return

    # Clear data in range L2:M, Odometer value in Engine Oil and Fallback date removal
    clear_range = 'L2:M'
    print(f"🧹 Clearing range {clear_range} in {worksheet_name}...")
    worksheet.batch_clear([clear_range])

    print("🔍 Starting data processing with date fallback search...")

    # 📥 Load main sheet
    print("📥 Loading worksheet data...")
    main_data = worksheet.get_all_values()
    print(f"✅ Loaded {len(main_data)-1} rows from worksheet")

    # Step 1: Collect unique (bus, date) and row mapping
    print("🔢 Mapping bus numbers and dates...")
    bus_date_map = defaultdict(set)
    row_map = {}  # maps row number → (bus, date)

    for i, row in enumerate(main_data[1:], start=2):
        try:
            date_str = row[1].strip()
            bus_num = row[4].strip()
            if date_str and bus_num:
                bus_date_map[bus_num].add(date_str)
                row_map[i] = (bus_num, date_str)
        except:
            continue

    print(f"✅ Found {len(bus_date_map)} unique bus numbers")
    print(f"✅ Found {len(row_map)} rows to process")

    # Step 2: Fetch each bus worksheet once, collect all data and find exact or closest earlier date
    print("\n🔍 Searching for exact dates or closest earlier dates...")
    bus_date_value_map = {}
    match_type_map = {}  # Tracks whether each match is exact or fallback
    fallback_date_map = {}  # Store the actual fallback date used
    success_count = 0
    fallback_count = 0

    for bus, dates in bus_date_map.items():
        print(f"📊 Processing bus {bus} ({len(dates)} dates)...")
        try:
            # Assuming Odometer_spreadsheet_SVP is defined in your notebook
            sheet = Odometer_spreadsheet_SVP.worksheet(bus)
            sheet_data = sheet.get_all_values()
            sheet_success = 0

            for date_str in dates:
                # Find exact match first
                exact_value = None
                for row in sheet_data[1:]:
                    try:
                        if len(row) > 12 and row[1].strip() == date_str:
                            exact_value = row[12]  # Column M
                            break
                    except:
                        continue

                if exact_value:
                    bus_date_value_map[(bus, date_str)] = exact_value
                    match_type_map[(bus, date_str)] = "exact"
                    fallback_date_map[(bus, date_str)] = ""  # No fallback date for exact matches
                    sheet_success += 1
                    success_count += 1
                else:
                    # Try to find closest earlier date
                    value, fallback_date = find_closest_date_value(sheet_data, date_str, value_column_index=12)
                    if value:
                        bus_date_value_map[(bus, date_str)] = value
                        match_type_map[(bus, date_str)] = "fallback"
                        fallback_date_map[(bus, date_str)] = fallback_date  # Store the fallback date used
                        sheet_success += 1
                        fallback_count += 1

            print(f"  ✅ Found {sheet_success}/{len(dates)} values for bus {bus}")

        except Exception as e:
            print(f"❌ Could not process sheet '{bus}': {str(e)}")
            continue

    print(f"\n✅ Found values for {success_count} exact date matches")
    print(f"✅ Found values for {fallback_count} fallback date matches")
    print(f"❌ Could not find values for {len(row_map) - (success_count + fallback_count)} dates")

    # Step 3: Update values into Column L of main sheet and fallback dates into Column M
    print(f"\n📝 Updating values and formatting in {worksheet_name}...")
    updated_count = 0

    # Prepare batch formatting requests for different match types
    exact_match_cells = []
    fallback_match_cells = []

    for row_num, (bus, date_str) in row_map.items():
        value = bus_date_value_map.get((bus, date_str), "")
        fallback_date = fallback_date_map.get((bus, date_str), "")

        if value:
            max_retries = 5
            retry_count = 0
            update_successful = False

            while not update_successful and retry_count < max_retries:
                try:
                    # Update cell value in column L (value)
                    worksheet.update_cell(row_num, 12, value)  # Column L = col 12

                    # Update column M with fallback date if applicable
                    worksheet.update_cell(row_num, 13, fallback_date)  # Column M = col 13

                    # Track which cells need which formatting
                    match_type = match_type_map.get((bus, date_str))
                    if match_type == "exact":
                        exact_match_cells.append(f"L{row_num}")
                    elif match_type == "fallback":
                        fallback_match_cells.append(f"L{row_num}")

                    update_successful = True
                    updated_count += 1
                    if updated_count % 10 == 0:
                        print(f"📊 Progress: Updated {updated_count}/{len(bus_date_value_map)} values")
                except Exception as e:
                    retry_count += 1
                    if "429" in str(e):
                        wait_time = (2 ** retry_count) + random.random()
                        print(f"⏳ Rate limit hit, waiting for {wait_time:.2f} seconds (retry {retry_count}/{max_retries})")
                        time.sleep(wait_time)
                    else:
                        print(f"❌ Failed to update row {row_num}: {e}")
                        break

            if not update_successful:
                print(f"❌ Failed to update row {row_num} after {max_retries} retries")

    # Apply formatting in batches (currently commented out)
    print("\n🎨 Applying formatting to distinguish match types...")
    # You can uncomment and implement batch formatting logic if needed.

    print(f"\n✅ COMPLETE for {worksheet_name}: Updated {updated_count} rows in column L and column M")
    print(f"📊 Summary: {success_count} exact matches (normal format), {fallback_count} fallback matches (italic and right-aligned)")
    print(f"❌ {len(row_map) - updated_count} rows could not be updated")

# Main execution
print("🚀 Starting to process all engine oil worksheets...")

# Process each worksheet in sequence
for worksheet_name in engine_oil_worksheets:
    process_worksheet(worksheet_name)

print("\n✅ All worksheets have been processed successfully!")

In [ ]:
# With fallback date display in M Column, For ATR_MAS_Engine OIl_ Best Code

# With fallback date display in M Column, Best Code

# Make sure to include these imports at the top of your code
import re
from datetime import datetime
import random
import time
from collections import defaultdict
import gspread


# Clear data in range L2:M, Odometer value in Engine Oil and Fallback date removal
clear_range_all_engine_oil_CH_BSIV_CH_MAS_atr = 'L2:M'
engine_oil_CH_BSIV_CH_MAS_ATR.batch_clear([clear_range_all_engine_oil_CH_BSIV_CH_MAS_atr])

# Function to parse custom date format 'DD,MMMyy' like '13,Apr24'
def parse_custom_date(date_str):
    if not date_str or date_str == '':
        return None

    pattern = r'(\d{1,2}),(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)(\d{2})'
    match = re.match(pattern, date_str)

    if match:
        day = int(match.group(1))
        month_str = match.group(2)
        year = int(match.group(3)) + 2000  # Assuming 20xx for the year

        month_map = {
            'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6,
            'Jul': 7, 'Aug': 8, 'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12
        }

        month = month_map[month_str]
        return datetime(year, month, day)

    return None

# Modified function to return both value and fallback date
def find_closest_date_value(sheet_data, target_date_str, value_column_index=12):
    # Parse all dates in the sheet
    dates = []
    parsed_dates = []
    values = []

    for row in sheet_data[1:]:  # Skip header
        try:
            if len(row) > value_column_index and len(row) > 1:
                date_str = row[1].strip()
                value = row[value_column_index]

                parsed_date = parse_custom_date(date_str)
                if parsed_date:
                    dates.append(date_str)
                    parsed_dates.append(parsed_date)
                    values.append(value)
        except:
            continue

    # Parse target date
    target_date = parse_custom_date(target_date_str)
    if not target_date:
        return None, None

    # First check for exact match
    for i, parsed_date in enumerate(parsed_dates):
        if parsed_date == target_date:
            return values[i], None  # Return value with no fallback date for exact matches

    # If no exact match, find closest earlier date
    closest_date = None
    closest_value = None
    max_date = None

    for i, parsed_date in enumerate(parsed_dates):
        if parsed_date < target_date and (max_date is None or parsed_date > max_date):
            max_date = parsed_date
            closest_date = dates[i]
            closest_value = values[i]

    return closest_value, closest_date  # Return both value and fallback date

print("🔍 Starting data processing with date fallback search...")

# 📥 Load main sheet
print("📥 Loading main sheet data...")
main_data = engine_oil_CH_BSIV_CH_MAS_ATR.get_all_values()
print(f"✅ Loaded {len(main_data)-1} rows from main sheet")

# Step 1: Collect unique (bus, date) and row mapping
print("🔢 Mapping bus numbers and dates...")
bus_date_map = defaultdict(set)
row_map = {}  # maps row number → (bus, date)

for i, row in enumerate(main_data[1:], start=2):
    try:
        date_str = row[1].strip()
        bus_num = row[4].strip()
        if date_str and bus_num:
            bus_date_map[bus_num].add(date_str)
            row_map[i] = (bus_num, date_str)
    except:
        continue

print(f"✅ Found {len(bus_date_map)} unique bus numbers")
print(f"✅ Found {len(row_map)} rows to process")

# Step 2: Fetch each bus worksheet once, collect all data and find exact or closest earlier date
print("\n🔍 Searching for exact dates or closest earlier dates...")
bus_date_value_map = {}
match_type_map = {}  # Tracks whether each match is exact or fallback
fallback_date_map = {}  # Store the actual fallback date used
success_count = 0
fallback_count = 0

for bus, dates in bus_date_map.items():
    print(f"📊 Processing bus {bus} ({len(dates)} dates)...")
    try:
        sheet = Odometer_spreadsheet_SVP.worksheet(bus)
        sheet_data = sheet.get_all_values()
        sheet_success = 0

        for date_str in dates:
            # Find exact match first
            exact_value = None
            for row in sheet_data[1:]:
                try:
                    if len(row) > 12 and row[1].strip() == date_str:
                        exact_value = row[12]  # Column M
                        break
                except:
                    continue

            if exact_value:
                bus_date_value_map[(bus, date_str)] = exact_value
                match_type_map[(bus, date_str)] = "exact"
                fallback_date_map[(bus, date_str)] = ""  # No fallback date for exact matches
                sheet_success += 1
                success_count += 1
            else:
                # Try to find closest earlier date
                value, fallback_date = find_closest_date_value(sheet_data, date_str, value_column_index=12)
                if value:
                    bus_date_value_map[(bus, date_str)] = value
                    match_type_map[(bus, date_str)] = "fallback"
                    fallback_date_map[(bus, date_str)] = fallback_date  # Store the fallback date used
                    sheet_success += 1
                    fallback_count += 1

        print(f"  ✅ Found {sheet_success}/{len(dates)} values for bus {bus}")

    except Exception as e:
        print(f"❌ Could not process sheet '{bus}': {str(e)}")
        continue

print(f"\n✅ Found values for {success_count} exact date matches")
print(f"✅ Found values for {fallback_count} fallback date matches")
print(f"❌ Could not find values for {len(row_map) - (success_count + fallback_count)} dates")

# Step 3: Update values into Column L of main sheet and fallback dates into Column M
print("\n📝 Updating values and formatting in main sheet...")
updated_count = 0

# Get the worksheet object for batch updates
worksheet = engine_oil_CH_BSIV_CH_MAS_ATR

# Prepare batch formatting requests for different match types
exact_match_cells = []
fallback_match_cells = []

for row_num, (bus, date_str) in row_map.items():
    value = bus_date_value_map.get((bus, date_str), "")
    fallback_date = fallback_date_map.get((bus, date_str), "")

    if value:
        max_retries = 5
        retry_count = 0
        update_successful = False

        while not update_successful and retry_count < max_retries:
            try:
                # Update cell value in column L (value)
                worksheet.update_cell(row_num, 12, value)  # Column L = col 12

                # Update column M with fallback date if applicable
                worksheet.update_cell(row_num, 13, fallback_date)  # Column M = col 13

                # Track which cells need which formatting
                match_type = match_type_map.get((bus, date_str))
                if match_type == "exact":
                    exact_match_cells.append(f"L{row_num}")
                elif match_type == "fallback":
                    fallback_match_cells.append(f"L{row_num}")

                update_successful = True
                updated_count += 1
                if updated_count % 10 == 0:
                    print(f"📊 Progress: Updated {updated_count}/{len(bus_date_value_map)} values")
            except Exception as e:
                retry_count += 1
                if "429" in str(e):
                    wait_time = (2 ** retry_count) + random.random()
                    print(f"⏳ Rate limit hit, waiting for {wait_time:.2f} seconds (retry {retry_count}/{max_retries})")
                    time.sleep(wait_time)
                else:
                    print(f"❌ Failed to update row {row_num}: {e}")
                    break

        if not update_successful:
            print(f"❌ Failed to update row {row_num} after {max_retries} retries")

# Apply formatting in batches (currently commented out)
print("\n🎨 Applying formatting to distinguish match types...")
# You can uncomment and implement batch formatting logic if needed.

print(f"\n✅ COMPLETE: Updated {updated_count} rows in column L and column M")
print(f"📊 Summary: {success_count} exact matches (normal format), {fallback_count} fallback matches (italic and right-aligned)")
print(f"❌ {len(row_map) - updated_count} rows could not be updated")
